In [1]:
import pandas as pd
import sqlite3
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# viewing data folder

! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [3]:
# making connection to imdb

conn = sqlite3.connect('../unzippedData/im.db')

In [4]:
MovieBudget = pd.read_csv('../unzippedData/tn.movie_budgets.csv')
# MovieBudget

In [5]:
# function for cleaning numbers column

def clean_number_series(Series):
    Series = Series.str.replace(",", "")
    Series = Series.str.replace("$", "")
    Series = Series.astype(int)
    return Series

# function for cleaning Column headers

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")
    
# function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

In [6]:
# Reformatted Movie Budget

clean_column_headers(MovieBudget)
MovieBudget['Worldwide Gross'] = clean_number_series(MovieBudget['Worldwide Gross'])
MovieBudget['Domestic Gross'] = clean_number_series(MovieBudget['Domestic Gross'])
MovieBudget['Production Budget'] = clean_number_series(MovieBudget['Production Budget'])


/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_9287/1531069200.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Series = Series.str.replace("$", "")


In [7]:
# Making Profitability Column and Revenue Budget Ratio Column

MovieBudget['Profitability'] = MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross'] - MovieBudget['Production Budget']
MovieBudget['Revenue Budget Ratio'] = (MovieBudget['Worldwide Gross'] + MovieBudget['Domestic Gross']) / MovieBudget['Production Budget']

# MovieBudget.head()

In [8]:
# filtered movies df by date and budget 

Movies_After_Date = 1990
Budget_Max = 17000000
Budget_Min = 0

MovieBudgetFiltered = MovieBudget[(MovieBudget['Production Budget'] < Budget_Max) & (MovieBudget['Production Budget'] > Budget_Min)]
MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
MovieBudgetFiltered.reset_index(drop=True, inplace=True)
# MovieBudgetFiltered[:50]
MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)
MovieBudgetFiltered= MovieBudgetFiltered[(MovieBudgetFiltered['Release Date']>Movies_After_Date)].reset_index(drop=True)
MovieBudgetFiltered['Movie'] = MovieBudgetFiltered['Movie'].str.replace('â', "'")
# MovieBudgetFiltered

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_9287/4186488630.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered.sort_values('Revenue Budget Ratio', ascending=False, inplace=True)
/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_9287/4186488630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MovieBudgetFiltered['Release Date'] = MovieBudgetFiltered['Release Date'].str[-4:].astype(int)


In [9]:
# Connecting to Movie Basics Table where genres reside

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""

movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df
movie_genres_df = pd.DataFrame([movie_basics_df['Primary Title'], movie_basics_df['Genres']])
movie_genres_df = movie_genres_df.transpose()
movie_genres_df.columns = movie_genres_df.columns.str.replace('Primary Title', 'Movie')
movie_genres_df = movie_genres_df.drop_duplicates('Movie')
# movie_genres_df

In [10]:
# merged data sets

Merged_df = MovieBudgetFiltered.merge(movie_genres_df, on = 'Movie', how='left')

# Merged_df

In [11]:
# Explode Datasets

MergedExploded_df = Merged_df.explode('Genres')
MergedExploded_df.dropna(inplace=True)
MergedExploded_df.columns = MergedExploded_df.columns.str.replace('Revenue Budget Ratio', 'RBR')

# MergedExploded_df.dropna(subset='Genres', inplace=True)

MergedExploded_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,RBR,Genres
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,Horror
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,Mystery
2,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,Thriller
14,12,2012,The Devil Inside,1000000,53262945,101759490,154022435,155.022435,Horror
21,90,2014,God's Not Dead,1150000,60755732,63777092,123382824,108.289412,Drama
...,...,...,...,...,...,...,...,...,...
2407,15,2013,My Last Day Without You,1000000,0,0,-1000000,0.000000,Comedy
2407,15,2013,My Last Day Without You,1000000,0,0,-1000000,0.000000,Drama
2407,15,2013,My Last Day Without You,1000000,0,0,-1000000,0.000000,Romance
2408,14,2014,Hidden Away,1000000,0,0,-1000000,0.000000,Drama


In [12]:
MergedExploded_df['Genres'].value_counts()

Drama          610
Comedy         280
Thriller       201
Horror         191
Romance        140
Crime          130
Action         123
Documentary    107
Mystery         95
Biography       84
Adventure       67
Sci-Fi          63
Family          47
Music           37
Fantasy         35
History         30
Sport           26
War             13
Animation       10
Musical          7
Western          7
News             1
Name: Genres, dtype: int64

In [13]:
R1_LookupTable = list(MergedExploded_df['Genres'].value_counts()[:10].index)
R1_LookupTable
MergedExploded_df = MergedExploded_df.loc[MergedExploded_df['Genres'].isin(R1_LookupTable)]
# FILTER['Genres'].value_counts()

In [14]:
MergedExploded_df['Genres'].value_counts()

Drama          610
Comedy         280
Thriller       201
Horror         191
Romance        140
Crime          130
Action         123
Documentary    107
Mystery         95
Biography       84
Name: Genres, dtype: int64

In [15]:
# Mean RBR by Genre
MergedExploded_df.groupby(by='Genres').mean()['RBR'].sort_values(ascending=False)

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_9287/352577621.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  MergedExploded_df.groupby(by='Genres').mean()['RBR'].sort_values(ascending=False)


Genres
Mystery        18.503032
Horror         13.561859
Thriller       10.330600
Romance         6.218651
Biography       5.905168
Drama           5.052561
Comedy          4.655731
Action          3.761849
Documentary     3.673962
Crime           2.666134
Name: RBR, dtype: float64

In [16]:
# Linear Regression and Anova
results = smf.ols(formula="RBR ~ C(Genres)", data=MergedExploded_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RBR   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     4.653
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           4.04e-06
Time:                        10:51:20   Log-Likelihood:                -9284.3
No. Observations:                1961   AIC:                         1.859e+04
Df Residuals:                    1951   BIC:                         1.864e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    3.7618      2.489      1.511      0.131      -1.120       8.644
C(Genres)[T.Biography]       2.1433      3.908      0.548      0.583      -5.520       9.807
C(Genres)[T.Comedy]          0.8939      2.986      0.299      0.765      -4.963       6.751
C(Genres)[T.Crime]          -1.0957      3.473     -0.316      0.752      -7.906       5.715
C(Genres)[T.Documentary]    -0.0879      3.650     -0.024      0.981      -7.245       7.070
C(Genres)[T.Drama]           1.2907      2.729      0.473      0.636      -4.061       6.642
C(Genres)[T.Horror]          9.8000      3.192      3.070      0.002       3.541      16.059
C(Genres)[T.Mystery]        14.7412      3.771      3.909      0.000       7.346      22.136
C(Genres)[T.Romance]         2.4568      3.412      0.720      0.472      -4.234       9.148
C(Genres)[T.Thriller]        6.5688      3.160      2.078      0.038       0.371      12.767
==============================================================================
Omnibus:                     4224.918   Durbin-Watson:                   0.199
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         14356567.016
Skew:                          18.663   Prob(JB):                         0.00
Kurtosis:                     420.507   Cond. No.                         14.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# Connecting to Persons Schema and Joining with Known For and Movie Basics Schemas

persons_query = """ 
    SELECT *
    FROM persons
    JOIN directors
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)

clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title'])
# joined_df['Primary Profession'] = clean_comma_series(joined_df['Primary Profession'])
# joined_df.columns = joined_df.columns.str.replace('Primary Profession', 'Profession')
joined_df.columns = joined_df.columns.str.replace('Primary Title', 'Movie')
joined_df.columns = joined_df.columns.str.replace('Primary Name', 'Name')
joined_df = joined_df.drop_duplicates(subset='Name')


MergedExploded_df = MergedExploded_df.merge(joined_df, on = 'Movie', how='left')


In [19]:
# Creating Top Three Genres Dataframe
Merged_df.dropna(inplace=True)
Merged_df = Merged_df.merge(joined_df, on = 'Movie', how='left')
TopThreeGenres_df = Merged_df.loc[(Merged_df['Genres'].apply(lambda x: 'Horror' in x)) | (Merged_df['Genres'].apply(lambda x: 'Mystery' in x))
             | (Merged_df['Genres'].apply(lambda x: 'Thriller' in x))].reset_index(drop=True)
TopThreeGenres_df.dropna(inplace=True)
TopThreeGenres_df[:50]

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,Revenue Budget Ratio,Genres,Name
0,80,2015,The Gallows,100000,22764410,41656474,64320884,644.208840,"[Horror, Mystery, Thriller]",Chris Lofing
1,12,2012,The Devil Inside,1000000,53262945,101759490,154022435,155.022435,[Horror],William Brent Bell
5,49,2017,Get Out,5000000,176040665,255367951,426408616,86.281723,"[Horror, Mystery, Thriller]",Jordan Peele
7,15,2010,The Last Exorcism,1800000,41034350,70165900,109400250,61.777917,"[Drama, Horror, Thriller]",Daniel Stamm
8,18,2012,Chernobyl Diaries,1000000,18119640,42411721,59531361,60.531361,"[Horror, Mystery, Thriller]",Bradley Parker
9,84,2014,Annabelle,6500000,84273813,256862920,334636733,52.482574,"[Horror, Mystery, Thriller]",John R. Leonetti
10,67,2013,The Purge,3000000,64473115,91266581,152739696,51.913232,"[Horror, Thriller]",James DeMonaco
12,69,2012,Sinister,3000000,48086903,87727807,132814710,45.271570,"[Horror, Mystery, Thriller]",Holly Bell
15,91,2018,Truth or Dare,3500000,41411015,95127344,133038359,39.010960,"[Horror, Romance, Thriller]",Jessica Cameron
16,91,2018,Truth or Dare,3500000,41411015,95127344,133038359,39.010960,"[Horror, Romance, Thriller]",Tom Boyle


In [20]:
TopThreeGenres_df.groupby(by='Name').mean()['Revenue Budget Ratio'].sort_values(ascending=False)[:50]

/var/folders/9n/kzdbb24d6718p004k3bbjtr40000gn/T/ipykernel_9287/1146644880.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TopThreeGenres_df.groupby(by='Name').mean()['Revenue Budget Ratio'].sort_values(ascending=False)[:50]


Name
Chris Lofing              644.208840
William Brent Bell        155.022435
Jordan Peele               86.281723
Daniel Stamm               61.777917
Bradley Parker             60.531361
John R. Leonetti           52.482574
James DeMonaco             51.913232
Holly Bell                 45.271570
Yun Xie                    39.010960
Tom Boyle                  39.010960
Suzette Reyes              39.010960
Junshu Huang               39.010960
Jessica Cameron            39.010960
Zhi-yu Ma                  39.010960
Dan Trachtenberg           36.073884
Darren Aronofsky           33.709338
Stiles White               30.831328
Graham Wright              27.742200
Stephen Susco              25.301333
Ann Tramonte               19.591680
Robert Eggers              18.740921
Huy Yaleng                 18.740921
P.C. Shekar                15.603440
Bernard Kordieh            15.072143
Sohee Myoung               15.072143
Wusi Liu                   15.072143
Laura Lau                  14.675

In [21]:
TopThreeGenres_df['Name'].value_counts()

Jason Stone                 2
Rich Varville               2
Hans Rodionoff              2
Emilio Belmonte             2
Stephen Milburn Anderson    2
                           ..
Jeremy Saulnier             1
Lynne Ramsay                1
Rob Zombie                  1
Christopher Macias          1
Tamar Simon Hoffs           1
Name: Name, Length: 360, dtype: int64